# FFT를 이용한 고속 다항식 곱셈 보고서

## 1. DFT, FFT, IFFT의 수식과 조건

이 보고서에서는 일반화된 형태로 FFT를 해석하고, 다항식 곱셈에 어떻게 응용되는지를 다룬다. 먼저 DFT의 정의부터 살펴보자.

### DFT의 정의

길이가 $n$인 복소수 벡터 $\mathbf{a} = (a_0, a_1, \dots, a_{n-1})$에 대해, DFT는 다음과 같은 수식을 따른다.

$$
A_k = \sum_{j=0}^{n-1} a_j \cdot \omega_n^{jk} \quad (0 \leq k < n)
$$

여기서 $\omega_n = e^{-2\pi i / n}$은 $n$번째 **단위근**이다.  
이때 $\omega_n^n = 1$을 만족하고, $\omega_n^k \neq 1$ for $0 < k < n$ 인 경우를 **primitive**한 단위근이라고 한다.

> **DFT가 성립하기 위한 조건**
> - 변환에 사용되는 $\omega_n$은 $n$번째 단위근이어야 한다.
> - 인덱스 $jk$는 모듈로 $n$에 따라 해석되므로 **인덱스 정렬과 순서**에 유의해야 한다.

> **단위근의 의미**  
> 복소평면 상에서 $\omega_n = e^{-2\pi i / n}$는 단위원의 원을 $n$등분하는 각도를 의미한다.  
> 즉, $\omega_n^k$는 반지름 1인 원에서 $k$번째 점을 가리키며, 이는 주기적 회전의 성질을 내포한다.

---
## 2. IFFT의 수식 유도 (역행렬 관점)

FFT는 다음과 같이 다항식을 점값으로 변환하는 DFT 행렬 곱 형태로 표현할 수 있다:

$P(x) = p_0 + p_1x + p_2x^2 + \cdots + p_{n-1}x^{n-1}$ 에 대해,

$$
\begin{bmatrix}
P(w^0) \\
P(w^1) \\
P(w^2) \\
\vdots \\
P(w^{n-1})
\end{bmatrix}
=
\begin{bmatrix}
1 & 1 & 1 & \cdots & 1 \\
1 & w & w^2 & \cdots & w^{n-1} \\
1 & w^2 & w^4 & \cdots & w^{2(n-1)} \\
\vdots & \vdots & \vdots & \ddots & \vdots \\
1 & w^{n-1} & w^{2(n-1)} & \cdots & w^{(n-1)^2}
\end{bmatrix}
\begin{bmatrix}
p_0 \\
p_1 \\
p_2 \\
\vdots \\
p_{n-1}
\end{bmatrix}
$$

여기서 $w = e^{\frac{2\pi i}{n}}$ 는 $n$번째 단위근이다. 위 행렬은 DFT 행렬 $\mathbf{F}_n$이라 부르며, 이를 통해 점값 표현으로의 변환이 이루어진다.

---

이제 IFFT는 이 DFT 행렬의 역행렬을 통해 계수로 되돌리는 연산이다:

$$
\begin{bmatrix}
p_0 \\
p_1 \\
p_2 \\
\vdots \\
p_{n-1}
\end{bmatrix}
=
\mathbf{F}_n^{-1}
\begin{bmatrix}
P(w^0) \\
P(w^1) \\
P(w^2) \\
\vdots \\
P(w^{n-1})
\end{bmatrix}
$$

그리고 $\mathbf{F}_n^{-1}$은 다음과 같이 나타낼 수 있다:

$$
\mathbf{F}_n^{-1} = \frac{1}{n}
\begin{bmatrix}
1 & 1 & 1 & \cdots & 1 \\
1 & w^{-1} & w^{-2} & \cdots & w^{-(n-1)} \\
1 & w^{-2} & w^{-4} & \cdots & w^{-2(n-1)} \\
\vdots & \vdots & \vdots & \ddots & \vdots \\
1 & w^{-(n-1)} & w^{-2(n-1)} & \cdots & w^{-(n-1)^2}
\end{bmatrix}
$$

즉, IFFT 역시 DFT 구조를 유지한 채, 단위근을 켤레 복소수로 바꾸고 전체에 $\frac{1}{n}$을 곱하는 방식으로 계산된다. 이는 DFT의 **conjugate transpose가 역행렬**이 됨을 의미한다.

---

## 3. 다항식 곱셈에서의 FFT 활용

두 다항식의 곱을 계산하려고 할 때, $A(x) = \sum_{i=0}^{n-1} a_i x^i$, $B(x) = \sum_{i=0}^{n-1} b_i x^i$ 라고 하자.  
이들의 곱 $C(x) = A(x) \cdot B(x)$는 최대 $2n-2$차가 되므로, FFT를 적용하기 위해 $n$을 $2$의 거듭제곱 크기로 padding한다.

### 계수 표현과 점값 표현

- **계수 표현**: 다항식의 $x^i$에 대한 계수들의 열
- **점값 표현**: 특정 $x_i$에 대해 $P(x_i)$를 평가한 값

> $d$차 다항식은 $d+1$개의 서로 다른 점을 통과하는 다항식이 유일하게 존재한다는 점에서 점값 표현은 중요한 의미를 가진다.

---

## 4. 평가(Evaluation)와 재귀 구조

다항식 $P(x)$를 짝수차수, 홀수차수 항으로 나누면 다음과 같이 표현할 수 있다:

$$
P(x) = P_{even}(x^2) + x \cdot P_{odd}(x^2)
$$
 
## 짝수, 홀수 함수의 대칭성 시각화 
![even vs odd](../image/even_odd.png)

### 재귀적 구조의 핵심

- $\deg(P_{even}) = \lfloor n/2 \rfloor$
- $\deg(P_{odd}) = \lfloor n/2 \rfloor$
- 평가할 점의 수 역시 절반으로 줄어들며, 이는 $\mathcal{O}(n \log n)$ 구조를 가능하게 만든다.

### 여기서 핵심 개념: **입력 x가 아니라 x^2**

각 단계에서 $x$의 제곱이 전달된다. 즉,

$$
P(x) = P_{even}(x^2) + x \cdot P_{odd}(x^2)
$$

이 되며, 이는 다음 단계의 even/odd 함수가 각각 다시 재귀적으로 $x^2$를 입력받게 된다.  
이 구조는 **단위근의 재귀적 보존 성질**과도 맞물린다.

---

## 5. 단위근의 재귀적 성질

$n$번째 단위근 $\omega_n$이 있을 때, $\omega_n^2$는 $\omega_{n/2}$에 해당한다.

왜냐하면:

$$
\omega_n = e^{\frac{2\pi i}{n}} \Rightarrow (\omega_n)^2 = e^{\frac{4\pi i}{n}} = e^{\frac{2\pi i}{n/2}} = \omega_{n/2}
$$

즉, 제곱 연산은 **각도를 2배**로 만들고, **분할 수는 절반**으로 줄어든다.  
이로 인해 $n$개의 단위근 중에서 **짝수 인덱스 단위근들**만으로 $n/2$개의 단위근이 재귀적으로 구성된다.

## 복소평면 상 단위근의 제곱 시각화
![roots of unity](../image/unity.png)

---

## 6. 최종 구현 코드

```python
# FFT 구현

import cmath 

def fft(array, input_solution):
    n = len(array)
    if n == 1:
        return array
    
    result = []
    array2, array3 = [], []

    for i in range(n // 2): 
        array2.append(array[2 * i])
    for j in range(n // 2):
        array3.append(array[2 * j + 1]) 

    squared_half_solution = []
    for i in range(n // 2):
        squared_half_solution.append(input_solution[i] ** 2) 

    P = fft(array2, squared_half_solution)
    Q = fft(array3, squared_half_solution)

    for i in range(n // 2):
        result.append(P[i] + input_solution[i] * Q[i])
    for i in range(n // 2):
        result.append(P[i] - input_solution[i] * Q[i])
    
    return result

array = [1, 2, 3, 4, 5, 6, 7, 8]

n = len(array)
w_list = [cmath.exp(2j * cmath.pi * i / n) for i in range(n)]

result = fft(array, w_list[:n//2])
print(result)
```

# FFT를 이용한 분할정복으로 다항식의 곱을 $\nlog n$으로 풀기

## 1. 다항식의 표현

다항식을 표현하는 방법은 두 가지가 있다.

### (1) 계수 표현 (Coefficient Representation)

다항식을 계수의 나열로 표현하는 방식이다.  
차수가 낮은 항부터 오름차순으로 정리하면 다음과 같이 나타낼 수 있다:

$$(a_0, a_1, a_2, \ldots, a_d)$$

이때, 이 표현은 다음과 같은 다항식을 의미한다:

$$
f(x) = a_0 + a_1x + a_2x^2 + \cdots + a_dx^d
$$

### (2) 점값 표현 (Value Representation)

다항식 위의 $d+1$개의 점 $(x_i, f(x_i))$를 저장하는 방식이다.  
이 점값 표현의 가장 큰 특징은 다음과 같다:

> **$d+1$개의 서로 다른 점을 모두 지나는 $d$차 다항식은 오직 하나만 존재한다.**

이 성질은 FFT를 기반으로 한 다항식 곱셈에서 핵심이 된다.

## 2. 다항식의 유일성 증명

$d=2$일 때를 예시로 들어 보자. 즉, 2차 다항식이 세 점을 지난다고 가정한다.  
이때 서로 다른 두 개의 2차 다항식 $P(x)$, $Q(x)$가 동일한 세 점 $(x_1, y_1), (x_2, y_2), (x_3, y_3)$을 지난다고 하자.

두 다항식의 차를 정의하면:

$$
R(x) = P(x) - Q(x)
$$

$R(x)$는 2차 이하의 다항식이다. 그런데:

$$
R(x_1) = R(x_2) = R(x_3) = 0
$$

즉, $R(x)$는 서로 다른 세 점에서 0이 된다.  
하지만 2차 이하의 다항식은 최대 두 개의 서로 다른 실근만 가질 수 있다.  
세 점에서 모두 0이 되는 것은 불가능하므로 모순이다.
따라서 서로 다른 $d+1$개의 점을 모두 지나는 $d$차 다항식은 유일하다.


In [ ]:
#fft 구현

import cmath 

def fft(array, input_solution):
    n = len(array)
    if n == 1:
        return array
    
    result = []
    array2, array3 = [], []

    for i in range(n // 2): 
        array2.append(array[2 * i])
    for j in range(n // 2):
        array3.append(array[2 * j + 1]) 

    squared_half_solution = []
    for i in range(n // 2):
        squared_half_solution.append(input_solution[i] ** 2) 

    P = fft(array2, squared_half_solution)
    Q = fft(array3, squared_half_solution)

    for i in range(n // 2):
        result.append(P[i] + input_solution[i] * Q[i])
    for i in range(n // 2):
        result.append(P[i] - input_solution[i] * Q[i])
    
    return result

array = [1, 2, 3, 4, 5, 6, 7, 8]

n = len(array)
w_list = [cmath.exp(2j * cmath.pi * i / n) for i in range(n)]

result = fft(array, w_list[:n//2])
print(result)

print(w_list[0])

In [ ]:
#정확한 수식 구현

from sympy import symbols, exp, I, pi, Rational, simplify

def fft(array, input_solution):
    n = len(array)
    if n == 1:
        return array
    
    result = []
    array2 = [array[2*i] for i in range(n//2)]
    array3 = [array[2*i+1] for i in range(n//2)]

    squared_half_solution = [simplify(w**2) for w in input_solution[:n//2]]

    P = fft(array2, squared_half_solution)
    Q = fft(array3, squared_half_solution)

    for i in range(n//2):
        result.append(simplify(P[i] + input_solution[i] * Q[i]))
    for i in range(n//2):
        result.append(simplify(P[i] - input_solution[i] * Q[i]))

    return result

# 입력
n = 8
w_list = [exp(2 * pi * I * i / n) for i in range(n)]
array = [Rational(x) for x in [1,2,3,4,5,6,7,8]]

# 실행
result = fft(array, w_list[:n//2])
for val in result:
    print(val)